# Clase 8
- **Alumna:** Enya Quetzalli Gómez Rodríguez *(Eduardo Gómez Rodríguez)*
- **Profesora:** Olga Kolesnikova
- **Escuela:** Escuela Superior de Cómputo del IPN
- **Grupo:** 3CV9
- **Semestre:** 2020/2

## Similitud de Palabras
Para poder determinar la similud de una palabra con otroa, debemos de conocer el contexto con el que estas son utilizadas, de modo de podemos utilizar vectores n-dimensionales donde la n es el tamaño del vocabulario con el que contamos y donde la i-ésima dimensión corresponde a la frecuenca con la que aparecen las palabras del contexto de nuestra palabra. 
#### Ventana de contexto
La ventana de contexto, es la cantidad de la palabras que tomaremos a la izquierda y a la derecha de nuestra palabra objetivo, de forma que con dicho contexto lograremos obtener un vector n-dimensional para cada palabra con la que contemos. 
#### Determinación de similitud
Utilizaremos el coseno del vector de una palabra $A$, con el de una palabra $B$, el cuál mientras más ceranos sean los vectores más tendrá a 1 el resultado del conseno, de forma que es una forma de saber que dos palabras son similares entre sí.
$$
similaridad(A,B) = \dfrac{A \cdot B}{||A||*||B||}
$$
Finalmente ordenaremos todos los cosenos obtenidos de mayor a menor, es decir obtendremos una lista de palabras ordenadas de mayor a menor similaridad con una palabra objetivo que hayamos elegido.

## Práctica
### Primero obtendremos nuestro texto normalizado

In [1]:
import nltk

In [2]:
def clean_html(txt):
	from bs4 import BeautifulSoup
	return BeautifulSoup(txt,'lxml').get_text().lower()

In [3]:
def split_text(txt):
	return txt.replace('/', ' ').replace('.', ' ').replace('-', ' ')

In [4]:
def delete_trash(txt):
	import re
	good = {'\n'}
	for i in "abcdefghijklmnopqrstuvwxyz áéíóúñü":
		good.add(i)
	ans = ""
	for c in txt:
		if c in good:
			ans += c
	return ans

In [5]:
def delete_stopwords(txt):
	from nltk.corpus import stopwords
	ans = []
	stp = stopwords.words()
	for w in txt:
		if w not in stp:
			ans.append(w)
	return ans

In [6]:
f = open("./c8_docs/e961024.htm", "r", encoding="utf8")
org_txt = f.read()
f.close()

text = delete_trash(split_text(clean_html(org_txt)))
tokens = delete_stopwords(nltk.word_tokenize(text))
print(tokens[:200])

['emod', 'htm', 'http', 'www', 'excelsior', 'mx', 'art', 'html', 'excelsior', 'editorial', 'jueves', 'octubre', 'epigrama', 'jorge', 'mansilla', 'torres', 'critica', 'miami', 'herald', 'presidente', 'ecuatoriano', 'autoproclamarse', 'loco', 'neoliberalismo', 'aplica', 'encomio', 'hace', 'mismo', 'país', 'manicomio', 'editorial', 'nota', 'siguiente', 'http', 'www', 'excelsior', 'mx', 'art', 'html', 'excelsior', 'editorial', 'jueves', 'octubre', 'hungría', 'rebelión', 'antiestalinista', 'oscar', 'gonzalez', 'lopez', 'curso', 'octubre', 'insurgencia', 'popular', 'comandada', 'estudiantes', 'intelectuales', 'obreros', 'partidarios', 'establecer', 'suelo', 'húngaro', 'régimen', 'socialista', 'libertad', 'creció', 'vertiginosamente', 'desembocar', 'auténtica', 'insurrección', 'armada', 'estructura', 'estalinista', 'surgido', 'consecuencia', 'resultados', 'primera', 'guerra', 'mundial', 'años', 'distancia', 'conviene', 'analizar', 'acontecimientos', 'otoño', 'magyar', 'dada', 'conducta', 'ado

### Obtener vocabulario
El vocabulario determinará la cantidad de palabras diferentes con las que contamos, por lo que nuestro vector de similitud deberá ser de este tamaño $n$

In [7]:
def get_vocabulary(tokens):
	vocabulary = sorted(set(tokens))
	return vocabulary

In [8]:
vocav = get_vocabulary(tokens)
print(vocav[:100])

['abad', 'abajo', 'abanderada', 'abandonado', 'abandonar', 'abandonarnos', 'abarcar', 'abascadocarranza', 'abastecer', 'abastecerá', 'abasteciera', 'abastecimiento', 'abasto', 'abatieran', 'abatir', 'abierta', 'abiertamente', 'abiertas', 'abierto', 'abocará', 'aboga', 'abogado', 'abogaron', 'aborda', 'abordaremos', 'abordarán', 'aborto', 'abracadabrante', 'abran', 'abrazarlo', 'abre', 'abren', 'abriendo', 'abriera', 'abrieron', 'abril', 'abrir', 'abrió', 'abrumado', 'abrupta', 'abrían', 'absolutamente', 'absolutismo', 'absolutizadas', 'absoluto', 'absorber', 'absorbería', 'absorción', 'abstecimiento', 'abstracciones', 'abstracción', 'absurdo', 'abundantes', 'abundar', 'abundó', 'aburrimiento', 'abusar', 'acaban', 'acabar', 'acabó', 'academia', 'académico', 'acantonadas', 'acaparó', 'acapulco', 'acariciando', 'acarrearía', 'acaso', 'acataría', 'acceder', 'accedido', 'accesible', 'acceso', 'accionaria', 'accionarias', 'accionario', 'acciones', 'accionistas', 'acción', 'aceite', 'acelerad

### Obtener ventanas de contexto
En este punto, obtendremos un diccionario que contenga las palabras que formen parte del contexto de cada palabra. 
Dado que será un proceso tardado, exportaremos este archivo en formato json para no tener que recalcular esta tarea en cada ocasión. 

In [9]:
def get_contexts(wordList, windowSize=4):
    contexts = dict()
    index = 0
    for index in range(len(wordList)):
        word = wordList[index]
        if word not in contexts:
            contexts[word] = []
        start = max(0, index - windowSize)
        end = min(index + windowSize, len(wordList) - 1)
        for i in range(start, end + 1):
            if i != index:
                contexts[word].append(wordList[i])
    return contexts

In [10]:
def dump_context(contexts):
    import json
    with open('./c8_docs/contexts.json', 'w') as outfile:
        json.dump(contexts, outfile, indent=4)

In [11]:
contexts = get_contexts(tokens)

In [12]:
print(sorted(contexts.items())[:10])

[('abad', ['propio', 'deshaciéndose', 'molesta', 'injerencia', 'emérito', 'guadalupe', 'contextos', 'ahora']), ('abajo', ['raúl', 'castellanos', 'explicó', 'pleno', 'tribuna', 'sorpresivo', 'voto', 'diferenciado', 'ingresos', 'minorías', 'empujadas', 'aún', 'debido', 'inmigrantes', 'ocupan', 'empleos']), ('abanderada', ['tómese', 'cuenta', 'aquellas', 'corrientes', 'poderoso', 'irresistible', 'dogma', 'privatización']), ('abandonado', ['lanzando', 'regreso', 'atribulados', 'feligreses', 'rebaño', 'atraídos', 'hedonismo', 'urbes']), ('abandonar', ['carácter', 'religioso', 'necesario', 'añadieron', 'posturas', 'intransigentes', 'confrontación', 'país']), ('abandonarnos', ['dejar', 'comer', 'tener', 'hijos', 'morirnos', 'gustavo', 'edmundo', 'magdalena']), ('abarcar', ['emitidos', 'mercados', 'prosperó', 'propósito', 'operaciones', 'derivadas', 'grupos', 'financieros']), ('abascadocarranza', ['respondió', 'prudente', 'panista', 'salvador', 'terció', 'discusión', 'corrupción', 'asentó']), 

### Obtener vector de similitud
Para ello haremos uso de la librería de *numpy* y *math*

In [13]:
import numpy
vectors = dict()
for target, context in contexts.items():
    cF = dict()
    for word in context:
        if word not in cF:
            cF[word] = 0
        cF[word] += 1
    l = numpy.zeros(len(vocav))
    for i in range(len(vocav)):
        word = vocav[i]
        if word in cF:
            l[i] = cF[word]
    vectors[target] = l

In [14]:
ans = []
v1 = vectors["acero"]
for word in vocav:
    v2 = vectors[word]
    cosine = numpy.dot(v1, v2) / numpy.sqrt(numpy.sum(v1 ** 2)) / numpy.sqrt(numpy.sum(v2 ** 2))
    ans.append((word, cosine))
ans.sort(key = lambda x: x[1], reverse=True)

In [15]:
with open("./c8_docs/similarity.txt", "w", encoding="utf8") as f:
    for par in ans:
        f.write(F"{par[0]} {par[1]}\n")

In [16]:
for par in ans[:30]:
    print(F"{par[0]} {par[1]}\n")

acero 0.9999999999999999

pesado 0.7499999999999999

trasladado 0.7499999999999999

cable 0.5303300858899106

cobre 0.49999999999999994

desmaterializado 0.37499999999999994

microprocesadores 0.37499999999999994

tubos 0.37499999999999994

vacío 0.35355339059327373

cables 0.24999999999999997

compromete 0.24999999999999997

penetración 0.24999999999999997

quede 0.24253562503633294

sostenible 0.22360679774997894

sino 0.21465741571734478

cubrir 0.2080125735844609

desplome 0.2080125735844609

paralelamente 0.2041241452319315

evitando 0.17677669529663687

finalizar 0.17677669529663687

crean 0.16666666666666666

externo 0.16439898730535726

depresión 0.15811388300841894

frijol 0.15811388300841894

consumo 0.14433756729740643

cosas 0.1424507205745474

tan 0.14048787173725408

luz 0.1386750490563073

alrededor 0.1360827634879543

producción 0.13514747567989718

